In [1]:
import numpy as np
import sys
sys.path.append('../')
from model.erroneousChoice import  erroneousChoice
from kernel import jaxrbf
from utility import  paramz
import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rc('xtick', labelsize=12) 
matplotlib.rc('ytick', labelsize=12)     

# Example using a benchmark problem from optimisation
We will use the benchmark DTLZ1, implementation from Botorch, to generate high-dimensional choice data. The number of covariates is equal to 5 and the number of utilities to 3.
We simulate the case where the subject has to select the best objects in a set of dimension $|A|=10$. Overall we will simulate 300 choices. We will use the first 200 as training set and the remainining 100 as test set.


In [16]:
import torch
from botorch.utils.multi_objective import is_non_dominated
from botorch.test_functions.multi_objective import DTLZ1

def is_pareto(X):
    return is_non_dominated(torch.from_numpy(X),deduplicate=False)


dimx=5
num_objectives=3
problem = DTLZ1(dimx,num_objectives=num_objectives,negate=True)
def fun(x):
    return np.array(problem(torch.from_numpy(x)))
bounds = [[0,1],[0,1],[0,1],[0,1],[0,1]]

#generate CA RA sets
def make_CA_RA(x, y, rows=[]):
    if len(rows)==0:
        rows=np.arange(x.shape[0])
    acc = rows[is_pareto(y)]
    rej = rows[~ is_pareto(y)]
    return acc, rej

def make_observations(X, fun, nA, dimA):
    CA=[]
    RA=[]   
    ix = 0
    for i in range(nA):
        rows = np.random.permutation(np.arange(X.shape[0]))[0:dimA]
        x=X[rows,:]
        y=fun(x)
        acc,rej=make_CA_RA(x, y, rows)
        if len(acc)>0:
            CA.append(acc)
        else:
            CA.append([])
        if len(acc)<dimA:
            RA.append(rej)
        else:
            RA.append([])
        ix = ix+1
    return CA, RA


#generate data
np.random.seed(1)

# we randomly generate objects
n = 100 # number of objects
X =np.vstack(bounds)[:,0]+np.random.rand(n,len(bounds))*(np.vstack(bounds)[:,1]-np.vstack(bounds)[:,0])

# we randomly generate choice data
nA = 300
dimA = 10
CA, RA = make_observations(X, fun, nA, dimA)

#train-test split
n_tr=200
indp = np.random.permutation(nA)
#trainining
CA_tr=[CA[i] for i in indp[0:n_tr]]
RA_tr=[RA[i] for i in indp[0:n_tr]]
#testing
CA_te=[CA[i] for i in indp[n_tr:]]
RA_te=[RA[i] for i in indp[n_tr:]]

In [18]:
latent_dim = num_objectives #equal to the true number of objectives
#data
data={'X': X,
      'CA': CA_tr,
      'RA': RA_tr,
      'dimA':dimA
          }

# define kernel: RBF 
Kernel = jaxrbf.RBF

#initial value for the hyperparameters of the kernel
params = {}
for i in range(latent_dim):
    params['lengthscale_'+str(i)]={'value':1.0*np.ones(data["X"].shape[1],float), 
                                'range':np.vstack([[0.1, 3.0]]*data["X"].shape[1]),
                                'transform': paramz.logexp()}
    params['variance_'+str(i)]   ={'value':np.array([3]), 
                                    'range':np.vstack([[1.0, 200.0]]),
                                    'transform': paramz.logexp()}
# define choice model 
model = erroneousChoice(data,Kernel,params,latent_dim)

# compute variational inference and estimate hyperparameters
model.optimize_hyperparams(niterations=4000,kernel_hypers_fixed=False)
print(model.params)


34984.36096971312
49380.26074503088
34184.91801321277
1536.7819128608805


  0%|          | 0/4000 [00:00<?, ?it/s]

27472.383906091047
1536.7819128608805


100%|██████████| 4000/4000 [25:43<00:00,  2.59it/s]


{'lengthscale_0': {'value': array([0.36590104, 0.65349102, 1.54004562, 5.08602279, 1.86262396]), 'range': array([[0.1, 3. ],
       [0.1, 3. ],
       [0.1, 3. ],
       [0.1, 3. ],
       [0.1, 3. ]]), 'transform': <utility.paramz.logexp object at 0x7fc4523524c0>}, 'variance_0': {'value': array([93.70290377]), 'range': array([[  1., 200.]]), 'transform': <utility.paramz.logexp object at 0x7fc4523b7be0>}, 'lengthscale_1': {'value': array([0.31057292, 0.08152722, 0.62937835, 0.86296821, 0.15523969]), 'range': array([[0.1, 3. ],
       [0.1, 3. ],
       [0.1, 3. ],
       [0.1, 3. ],
       [0.1, 3. ]]), 'transform': <utility.paramz.logexp object at 0x7fc4523b7a90>}, 'variance_1': {'value': array([37.24537701]), 'range': array([[  1., 200.]]), 'transform': <utility.paramz.logexp object at 0x7fc4523b7ee0>}, 'lengthscale_2': {'value': array([0.19600908, 0.14324927, 1.73730068, 0.24783269, 0.8837354 ]), 'range': array([[0.1, 3. ],
       [0.1, 3. ],
       [0.1, 3. ],
       [0.1, 3. ],
  

NameError: name 'Xpred' is not defined

In [19]:
# predicted samples
predictions = model.predict_VI(X)
#it returns the joint mean (predictions[0]) and joint covariance matrix (predictions[1]) 
#for the latent utilities. 
F = predictions[0]
F = F.reshape(latent_dim,X.shape[0]).T# these are the expected utility as a matrix: num_X times latent_dim

In [21]:
#Compute accuracy on test set
Pred=[]
YTrue=[]
for ii in range(0,len(CA_te)):
    if len(RA_te[ii])>0:
        Pred.append(is_non_dominated(torch.from_numpy(np.vstack([F[CA_te[ii]],
               F[RA_te[ii]]
              ]))))
        YTrue.append(np.hstack([np.ones(len(CA_te[ii])),np.zeros(len(RA_te[ii]))]))
    else:
        Pred.append(is_non_dominated(torch.from_numpy(np.array(F[CA_te[ii]]))))
        YTrue.append(np.hstack([np.ones(len(CA_te[ii]))]))
acc = len(np.where(abs(np.vstack(YTrue).astype(int)-np.vstack(Pred))==0)[0])/np.vstack(YTrue).size
print("accuracy=",acc)

0.916


Note that, in the above code, we used the expected utility to choose predict the subject's choice for the test set. A better way to perform this computation is to compute the probability that a subset of $A_k$ is the pareto set and for instance return the subset which has the highest probability to be undominated.